TO PRE-PROCESSING DATA
1. Min Max Scaler
2. Standard Scaler

In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StandardScaler

In [3]:
spark = SparkSession.builder.appName('Tutor_6_Preprocessing').getOrCreate()

In [14]:
training = spark.read.csv('data/test1.csv',header=True,inferSchema=True)

VectorAssembler() combine multiple feature columns into 
a single vector column that can be passed to ML algo for training.


In [15]:
inpCols = ['Age','Experience']
vct_assemb = VectorAssembler(inputCols=inpCols, outputCol='independent_feature')

In [19]:
assemb_df = vct_assemb.transform(training)
assemb_df.show()

+---------+---+----------+------+-------------------+
|     Name|Age|Experience|Salary|independent_feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [20]:
final_data = assemb_df.select('independent_feature', 'Salary')
final_data.show()

+-------------------+------+
|independent_feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



Linear Regression (Hồi quy tuyến tính)

In [18]:
from pyspark.ml.regression import LinearRegression

In [37]:
# Training test split
# Split Data into train_data (75%) and test_data (25%)
# Fit() to takes train_data and calculate the coefficient

train_data, test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='independent_feature', labelCol='Salary')
regressor = regressor.fit(train_data)

The regression format: y= w1x1 + w2x2 +...+ wn.xn + b
- y is the dependent var
- x is the independent feature
- w is the coefficient of each feature
- b is the intercept 

In [38]:
# coefficient: là trọng số đc gán cho mỗi giá trị trong 'indepen_feature'
# Từ đó, dự doán các giá trị trong dependent var ('Salary')
regressor.coefficients

DenseVector([-323.2867, 1696.8066])

In [39]:
# intercept: is the predicted val of dependent var ('Salary')
# when all the indenpendent var ('indepen_feature') = 0
regressor.intercept

22295.299605312008

In [51]:
### Prediction
pred_results = regressor.evaluate(test_data)

In [52]:
pred_results.predictions.show()

+-------------------+------+------------------+
|independent_feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|17203.085755292603|
+-------------------+------+------------------+

